In [1]:
%load_ext autoreload
%autoreload 2

import os
import itertools
import pickle
from os.path import join
from copy import deepcopy
from pdb import set_trace

import numpy as np
import pandas as pd
import mne
from IPython.display import display, clear_output

import deepbci as dbci
from deepbci.data_utils.load_files import load_openbci_v5, load_openbci_v4
from deepbci.data_utils.build_data import utils as bdtils

# v5

In [2]:
v5_filepath = join('test-data', 'v5', 'OpenBCI-RAW-dropped-samps.txt')

eeg_df, time_df = load_openbci_v5(v5_filepath)

NameError: name 'FS' is not defined

In [117]:
eeg_df

,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3
0,-200.036029,67.239661,-95.786318,-30.304409
1,-427.801546,23.830643,45.443524,79.016606
2,-645.138352,22.353382,-84.626456,139.219646
3,-608.453674,121.187718,-17.083863,26.772074
4,-425.479068,76.982101,66.962909,11.373036
...,...,...,...,...
12231,-4248.571228,-834.773719,140.870812,3664.473653
12232,-4426.093060,-660.404624,-35.270996,3824.939671
12233,-4556.593129,-527.039791,-100.012405,3946.985696
12234,-4788.106026,-580.666217,109.818422,4113.302787


In [118]:
time_df

,Sample Index,Timestamp (Formatted)
0,2.0,13:15:26.050
1,2.0,13:15:26.050
2,3.0,13:15:26.065
3,3.0,13:15:26.065
4,4.0,13:15:26.066
...,...,...
12231,98.0,13:16:27.312
12232,99.0,13:16:27.313
12233,99.0,13:16:27.313
12234,100.0,13:16:27.327


## Tests

For whatever reason the new OpenBCI GUI v5 always starts at sample 2 meaning the first 3 samples are always dropped.

Pattern for conversion is as follows: col 1) plus previous index, col 2) multiply by 2.



    0 -> 0, 0
    1 -> 1, 2
    2 -> 3, 4
    3 -> 5, 6
    ...
    100 -> 199, 200


In [80]:
data_len = len(time_df)
always_missing = 3
dropped = 22
fs = 200 + 1 

assert (data_len + always_missing + dropped) % fs == 0

In [81]:
_, counts = np.unique(time_df['Sample Index'].values, return_counts=True)

np.unique(counts, return_counts=True)

(array([ 60, 120, 122]), array([ 1, 12, 88]))

In [82]:
# def v5_to_v4_index(index):
    
#     fs = 200
#     half_fs = int(200 / 2)
#     index_map = [
#         dict(zip(np.arange(1, half_fs+1), np.arange(1, fs, 2))),
#         dict(zip(np.arange(1, half_fs+1), np.arange(2, fs+1, 2))),
#     ]
#     def get_mapped_values():
#         first_map = index_map[0][curr_vals[0]]
#         second_map =  index_map[1][curr_vals[1]]
# #         print(first_map, second_map)
#         return first_map, second_map
    
#     idx = 0
#     new_idx = []
#     while idx < len(index):
#         start = idx
#         end = start + 2
#         curr_vals = index[start:end]
        
#         print(idx, start, end, index[start:end])
        
#         if curr_vals[0] == curr_vals[1]:
#             mappings = get_mapped_values()
#             new_idx.extend(mappings)
#             idx += 2
#         # zero speical case
#         elif curr_vals[0] == 0 and curr_vals[1] == 1:
#             new_idx.append(0)
#             idx += 1
#         else:
#             first_map = index_map[0][curr_vals[0]]
#             new_idx.append(first_map)
#             idx += 1
            
#             print(f"Dropped sample:")
#             print(f"\tIndexes: {start}-{end}")
#             print(f"\tValues: {curr_vals[0]}-{curr_vals[1]}")
#             print(f"\tMapped Values: {first_map}-{index_map[0][curr_vals[1]]}")
        
#     return np.array(new_idx)

# new_idx = v5_to_v4_index(time_df['Sample Index'].values.astype('int'))
# assert len(new_idx) == len(time_df)

0 0 2 [2 2]
2 2 4 [3 3]
4 4 6 [4 4]
6 6 8 [5 5]
8 8 10 [6 6]
10 10 12 [7 7]
12 12 14 [8 8]
14 14 16 [9 9]
16 16 18 [10 10]
18 18 20 [11 11]
20 20 22 [12 12]
22 22 24 [13 13]
24 24 26 [14 14]
26 26 28 [15 15]
28 28 30 [16 16]
30 30 32 [17 17]
32 32 34 [18 18]
34 34 36 [19 19]
36 36 38 [20 20]
38 38 40 [21 21]
40 40 42 [22 22]
42 42 44 [23 23]
44 44 46 [24 24]
46 46 48 [25 25]
48 48 50 [26 26]
50 50 52 [27 27]
52 52 54 [28 28]
54 54 56 [29 29]
56 56 58 [30 30]
58 58 60 [31 31]
60 60 62 [32 32]
62 62 64 [33 33]
64 64 66 [34 34]
66 66 68 [35 35]
68 68 70 [36 36]
70 70 72 [37 37]
72 72 74 [38 38]
74 74 76 [39 39]
76 76 78 [40 40]
78 78 80 [41 41]
80 80 82 [42 42]
82 82 84 [43 43]
84 84 86 [44 44]
86 86 88 [45 45]
88 88 90 [46 46]
90 90 92 [47 47]
92 92 94 [48 48]
94 94 96 [49 49]
96 96 98 [50 50]
98 98 100 [51 51]
100 100 102 [52 52]
102 102 104 [53 53]
104 104 106 [54 54]
106 106 108 [55 55]
108 108 110 [56 56]
110 110 112 [57 57]
112 112 114 [58 58]
114 114 116 [59 59]
116 116 118 [60 60]

10673 10673 10675 [23 23]
10675 10675 10677 [24 24]
10677 10677 10679 [25 25]
10679 10679 10681 [26 26]
10681 10681 10683 [27 27]
10683 10683 10685 [28 28]
10685 10685 10687 [29 29]
10687 10687 10689 [30 30]
10689 10689 10691 [31 31]
10691 10691 10693 [32 32]
10693 10693 10695 [33 33]
10695 10695 10697 [34 34]
10697 10697 10699 [35 35]
10699 10699 10701 [36 36]
10701 10701 10703 [37 37]
10703 10703 10705 [38 38]
10705 10705 10707 [39 39]
10707 10707 10709 [40 40]
10709 10709 10711 [41 41]
10711 10711 10713 [42 42]
10713 10713 10715 [43 43]
10715 10715 10717 [44 44]
10717 10717 10719 [45 45]
10719 10719 10721 [46 46]
10721 10721 10723 [47 47]
10723 10723 10725 [48 48]
10725 10725 10727 [49 49]
10727 10727 10729 [50 50]
10729 10729 10731 [51 51]
10731 10731 10733 [52 52]
10733 10733 10735 [53 53]
10735 10735 10737 [54 54]
10737 10737 10739 [55 55]
10739 10739 10741 [56 56]
10741 10741 10743 [57 57]
10743 10743 10745 [58 58]
10745 10745 10747 [59 59]
10747 10747 10749 [60 60]
10749 10749 

In [119]:
odds = np.insert(np.arange(1, fs, 2), 0, 0)
evens = np.insert(np.arange(2, fs+1, 2), 0, 0)
key = np.arange(0, half_fs+1)
dict(zip(key, odds))

{0: 0,
 1: 1,
 2: 3,
 3: 5,
 4: 7,
 5: 9,
 6: 11,
 7: 13,
 8: 15,
 9: 17,
 10: 19,
 11: 21,
 12: 23,
 13: 25,
 14: 27,
 15: 29,
 16: 31,
 17: 33,
 18: 35,
 19: 37,
 20: 39,
 21: 41,
 22: 43,
 23: 45,
 24: 47,
 25: 49,
 26: 51,
 27: 53,
 28: 55,
 29: 57,
 30: 59,
 31: 61,
 32: 63,
 33: 65,
 34: 67,
 35: 69,
 36: 71,
 37: 73,
 38: 75,
 39: 77,
 40: 79,
 41: 81,
 42: 83,
 43: 85,
 44: 87,
 45: 89,
 46: 91,
 47: 93,
 48: 95,
 49: 97,
 50: 99,
 51: 101,
 52: 103,
 53: 105,
 54: 107,
 55: 109,
 56: 111,
 57: 113,
 58: 115,
 59: 117,
 60: 119,
 61: 121,
 62: 123,
 63: 125,
 64: 127,
 65: 129,
 66: 131,
 67: 133,
 68: 135,
 69: 137,
 70: 139,
 71: 141,
 72: 143,
 73: 145,
 74: 147,
 75: 149,
 76: 151,
 77: 153,
 78: 155,
 79: 157,
 80: 159,
 81: 161,
 82: 163,
 83: 165,
 84: 167,
 85: 169,
 86: 171,
 87: 173,
 88: 175,
 89: 177,
 90: 179,
 91: 181,
 92: 183,
 93: 185,
 94: 187,
 95: 189,
 96: 191,
 97: 193,
 98: 195,
 99: 197,
 100: 199}

In [120]:
dict(zip(key, evens))

{0: 0,
 1: 2,
 2: 4,
 3: 6,
 4: 8,
 5: 10,
 6: 12,
 7: 14,
 8: 16,
 9: 18,
 10: 20,
 11: 22,
 12: 24,
 13: 26,
 14: 28,
 15: 30,
 16: 32,
 17: 34,
 18: 36,
 19: 38,
 20: 40,
 21: 42,
 22: 44,
 23: 46,
 24: 48,
 25: 50,
 26: 52,
 27: 54,
 28: 56,
 29: 58,
 30: 60,
 31: 62,
 32: 64,
 33: 66,
 34: 68,
 35: 70,
 36: 72,
 37: 74,
 38: 76,
 39: 78,
 40: 80,
 41: 82,
 42: 84,
 43: 86,
 44: 88,
 45: 90,
 46: 92,
 47: 94,
 48: 96,
 49: 98,
 50: 100,
 51: 102,
 52: 104,
 53: 106,
 54: 108,
 55: 110,
 56: 112,
 57: 114,
 58: 116,
 59: 118,
 60: 120,
 61: 122,
 62: 124,
 63: 126,
 64: 128,
 65: 130,
 66: 132,
 67: 134,
 68: 136,
 69: 138,
 70: 140,
 71: 142,
 72: 144,
 73: 146,
 74: 148,
 75: 150,
 76: 152,
 77: 154,
 78: 156,
 79: 158,
 80: 160,
 81: 162,
 82: 164,
 83: 166,
 84: 168,
 85: 170,
 86: 172,
 87: 174,
 88: 176,
 89: 178,
 90: 180,
 91: 182,
 92: 184,
 93: 186,
 94: 188,
 95: 190,
 96: 192,
 97: 194,
 98: 196,
 99: 198,
 100: 200}

In [125]:
def v5_to_v4_index(index):
    new_idx = []
    prev_idx = None
    fs = 200
    half_fs = int(200 / 2)
    
    odds = np.insert(np.arange(1, fs, 2), 0, 0)
    evens = np.insert(np.arange(2, fs+1, 2), 0, 0)
    key = np.arange(0, half_fs+1)
    index_map = [
        dict(zip(key, odds)),
        dict(zip(key, evens)),
    ]

    for i, idx in enumerate(index):
        
#         print(i, idx, prev_idx)
        
        if idx == prev_idx:
            use_map = 1
        elif prev_idx == None or idx == 0:
            use_map = 0
        elif abs(idx - prev_idx) != 1:
            use_map = 0
            # does not include the current mapped value so 1 must be subtracted
            diff = index_map[use_map][idx] - new_idx[i-1] - 1
            print(f"Dropped samples: {diff}")
            print(f"\tIndexes: {i-1}-{i}")
            print(f"\tValues: {prev_idx}-{idx}")
            print(f"\tMapped values: [{new_idx[i-1]}, {index_map[use_map][idx]})")
        else:
            use_map = 0
            
        new_idx.append(index_map[use_map][idx])
        prev_idx = idx
    return np.array(new_idx)

new_idx= v5_to_v4_index(time_df['Sample Index'].values.astype('int'))
assert len(new_idx) == len(time_df)

Dropped samples: 2
	Indexes: 3066-3067
	Values: 27-29
	Mapped values: [54, 57)
Dropped samples: 4
	Indexes: 3108-3109
	Values: 49-52
	Mapped values: [98, 103)
Dropped samples: 2
	Indexes: 3112-3113
	Values: 53-55
	Mapped values: [106, 109)
Dropped samples: 10
	Indexes: 3170-3171
	Values: 83-89
	Mapped values: [166, 177)
Dropped samples: 4
	Indexes: 3188-3189
	Values: 97-100
	Mapped values: [194, 199)


In [110]:
time_df['Sample Index'] = new_idx
time_df

,Sample Index,Timestamp (Formatted)
0,3,13:15:26.050
1,4,13:15:26.050
2,5,13:15:26.065
3,6,13:15:26.065
4,7,13:15:26.066
...,...,...
12231,196,13:16:27.312
12232,197,13:16:27.313
12233,198,13:16:27.313
12234,199,13:16:27.327


In [111]:
time_df.shape

(12236, 2)

In [112]:
fill = bdtils.OpenBCISampleFill(eeg_df.values, time_df.values, fs=200)
midx, miter_pos, miters = fill()

┌────────────────────────────────────────────┐
│WARNING: Not filling samples before index 3!│
└────────────────────────────────────────────┘


In [113]:
(len(midx)+3) / 201

61.0

In [114]:
miters

{3066: {'start': 3066,
  'start_offset': 3066,
  'end_offset': 3068,
  'siter': 54,
  'eiter': 57,
  'data': [array([   7.  ,   90.3 ,   52.43, -187.5 ]),
   array([   3.06,  120.08,   67.65, -251.71])],
  'utils': array([[55, '13:15:41.414'],
         [56, '13:15:41.414']], dtype=object)},
 3108: {'start': 3108,
  'start_offset': 3110,
  'end_offset': 3114,
  'siter': 98,
  'eiter': 103,
  'data': [array([  42.98,  100.33,   63.16, -240.78]),
   array([  32.68,  113.16,   68.91, -261.34]),
   array([  22.38,  126.  ,   74.65, -281.9 ]),
   array([  12.08,  138.83,   80.4 , -302.46])],
  'utils': array([[99, '13:15:41.651'],
         [100, '13:15:41.651'],
         [101, '13:15:41.651'],
         [102, '13:15:41.651']], dtype=object)},
 3112: {'start': 3112,
  'start_offset': 3118,
  'end_offset': 3120,
  'siter': 106,
  'eiter': 109,
  'data': [array([  10.59,  105.45,   59.85, -196.06]),
   array([   2.96,   90.12,   50.55, -150.95])],
  'utils': array([[107, '13:15:41.697'],
       

# v4

In [41]:
v4_filepath = join('test-data', 'v4', 'OpenBCI-RAW-2020-02-17_13-36-34.txt')

eeg_df, time_df = load_openbci_v4(v4_filepath)

In [42]:
eeg_df

,1,2,3,4
0,2468.66,992.40,-15686.28,271.67
1,2648.67,1681.96,-15686.28,864.21
2,3011.30,1885.38,-15686.28,1077.42
3,2957.38,1839.76,-15686.28,1075.66
4,2699.75,1697.10,-15686.28,977.42
...,...,...,...,...
40396,-340.25,-512.07,-2720.06,-875.82
40397,-336.33,-518.38,-2722.51,-869.82
40398,-347.22,-542.54,-2734.10,-878.00
40399,-348.67,-544.68,-2737.06,-879.99


In [43]:
time_df

,0,8
0,0,13:36:34.749
1,1,13:36:34.764
2,2,13:36:34.764
3,3,13:36:34.768
4,4,13:36:34.768
...,...,...
40396,196,13:39:56.726
40397,197,13:39:56.742
40398,198,13:39:56.742
40399,199,13:39:56.743
